In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

데이터 셋 로드 (TFDS에서 가져온 EMNIST)

In [ ]:
(etfds_train, etfds_test), etfds_info = tfds.load("emnist", split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
print(etfds_info)

def normalize_img_ascii_lable(image, label):
    ascii_Lb = label
    if ascii_Lb >= 10:
        ascii_Lb += 8
    elif ascii_Lb >= 43:
        ascii_Lb += 6
    ascii_Lb += 15
    return tf.cast(image, tf.float32) / 255., ascii_Lb

etfds_train = etfds_train.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_train = etfds_train.cache()
etfds_train = etfds_train.shuffle(etfds_info.splits['train'].num_examples)
etfds_train = etfds_train.batch(128)
etfds_train = etfds_train.prefetch(tf.data.experimental.AUTOTUNE)

etfds_test = etfds_test.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_test = etfds_test.batch(128)
etfds_test = etfds_test.cache()
etfds_test = etfds_test.prefetch(tf.data.experimental.AUTOTUNE)

직접 만든 데이터 셋 로드

In [ ]:
datasetImgs = tf.data.Dataset.list_files('dataset/')

for f in datasetImgs.take(95):
    print(f.numpy())

def set_mydataset_img_label(imgNum, label):
    

In [ ]:
#datasetImgs = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255., validation_split=0.1)
#trainFG = datasetImgs.flow_from_directory(directory="../dataset/", target_size=(28,28), batch_size=10, class_mode='sparse', subset='training', shuffle=True, color_mode='grayscale')
#testFG = datasetImgs.flow_from_directory(directory="../dataset/", target_size=(28,28), batch_size=10, class_mode='sparse', subset='validation', shuffle=True, color_mode='grayscale')

#print(trainFG.classes)

#print(trainFG)

모델 세팅

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(95, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

훈련시키기

In [ ]:
model.fit(trainFG, epochs=5)
model.fit(etfds_train, epochs=5)


모델 저장하기

In [ ]:
model.save('spemnist.h5')

확인하기

In [ ]:
model.evaluate(etfds_test, verbose=2)
model.evaluate(testFG, verbose=2)